# Création de prompts pour le nommage et la description des clusters

In [2]:
Sys.setlocale("LC_ALL", "fr_CA.UTF-8")

# Library
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggtext)
library(tidytext)
library(ellipsellm)

# Load Data and Clustering Results
kmeans_result <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_kmeans.rds") # nolint
dfUsedForClustering <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_2025.rds")
df_clusters_composition <- readRDS("../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_composition.rds")

[1] "LC_CTYPE=fr_CA.UTF-8;LC_NUMERIC=C;LC_TIME=fr_CA.UTF-8;LC_COLLATE=fr_CA.UTF-8;LC_MONETARY=fr_CA.UTF-8;LC_MESSAGES=en_CA.UTF-8;LC_PAPER=en_CA.UTF-8;LC_NAME=C;LC_ADDRESS=C;LC_TELEPHONE=C;LC_MEASUREMENT=en_CA.UTF-8;LC_IDENTIFICATION=C"

In [3]:
cluster_names <- unique(dfUsedForClustering$cluster_name)
cluster_names <- cluster_names[order(cluster_names)]

prompts <- list()

for (cluster_value in cluster_names) {
  cluster_data <- dfUsedForClustering %>% filter(cluster_name == cluster_value)
  cluster_vars <- df_clusters_composition %>% filter(cluster_name == cluster_value)

  # Calculer quelques statistiques démographiques pour le cluster
  perc_femmes     <- mean(cluster_data$female, na.rm = TRUE) * 100
  perc_immigrants <- mean(cluster_data$immigrant, na.rm = TRUE) * 100
  perc_langEn     <- mean(cluster_data$langEn, na.rm = TRUE) * 100
  perc_langFr     <- mean(cluster_data$langFr, na.rm = TRUE) * 100
  perc_ses_languageOther <- mean(cluster_data$ses_languageOther, na.rm = TRUE) * 100
  perc_ethn_white <- mean(cluster_data$ses_ethn_White, na.rm = TRUE) * 100
  perc_hetero     <- mean(cluster_data$ses_hetero, na.rm = TRUE) * 100
  educ_mean       <- mean(cluster_data$educ, na.rm = TRUE)
  age_mean        <- mean(cluster_data$age, na.rm = TRUE) * 80
  revenu_moyen    <- mean(cluster_data$ses_income, na.rm = TRUE) * 225000

  # Génération du prompt de suggestion de noms
  prompt_description <- (paste0(
    "Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona ", cluster_value, " :\n",

    "Variables avec une moyenne significativement plus élevée que la moyenne de la population en général :\n",
    if (any(cluster_vars$color == "green")) {
      paste(cluster_vars[cluster_vars$color == "green", "variable"], collapse = "\n")
    } else {
      "Aucune variable n'a une moyenne significativement plus élevée que la moyenne de la population en général."
    },
    "\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\n",
    if (any(cluster_vars$color == "red")) {
      paste(cluster_vars[cluster_vars$color == "red", "variable"], collapse = "\n")
    } else {
      "Aucune variable avec une moyenne significativement plus basse que la moyenne de la population en général."
    },
    "\n\nCe cluster est composé de \n", 
    "Femmes: ", round(perc_femmes, 1), "%, 
     Immigrants: ", round(perc_immigrants, 1), "%\n",
    "Âge: ", round(age_mean, 1), " ans, 
     Revenu: ", round(revenu_moyen, 1), " k$\n",
    "Langues: 
     En ", round(perc_langEn, 1), "%, 
     Fr ", round(perc_langFr, 1), "%, 
     Autres: ", round(perc_ses_languageOther, 1), "%\n",
    "Édu: ", round(educ_mean, 1), " (0=bas, 1=élevé), 
     Ethnie (Blanc): ", round(perc_ethn_white, 1), "%\n",
    "Orientation (Hétéro): ", round(perc_hetero, 1), "%\n"
  ))

  # Stocker le prompt dans la liste
  prompts[[cluster_value]] <- prompt_description
}

Warning message in mean.default(cluster_data$female, na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(cluster_data$immigrant, na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(cluster_data$langEn, na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(cluster_data$langFr, na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(cluster_data$ses_ethn_White, na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(cluster_data$ses_hetero, na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(cluster_data$educ, na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(cluster_data$age, na.rm = TRUE):
“argument is not numeric or logical: returning NA”
Warning message in mean.default(cluste

In [3]:
prompts

$`1`
[1] "Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 1 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"ses_age\", \"ses_languageEnglish\")\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nAucune variable avec une moyenne significativement plus basse que la moyenne de la population en général.\n\nCe cluster est composé de \nFemmes: NA%, \n     Immigrants: NA%\nÂge: NA ans, \n     Revenu: 116452 k$\nLangues: \n     En NA%, \n     Fr NA%, \n     Autres: 0%\nÉdu: NA (0=bas, 1=élevé), \n     Ethnie (Blanc): NA%\nOrientation (Hétéro): NA%\n"

$`2`
[1] "Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 2 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"ses_age\", \"ses_languageFrench\")\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"lifestyle_goHuntingFreq_numeric\", \"lifestyle_volunteeringFreq_numeric\", \"ses_languageEnglish\")\n\nCe cluster est composé de \nFemmes: NA%, \n     Immigrants: NA%\nÂge: NA ans, \n     Revenu: 104135.7 k$\nLangues: \n     En NA%, \n     Fr NA%, \n     Autres: 0.4%\nÉdu: NA (0=bas, 1=élevé), \n     Ethnie (Blanc): NA%\nOrientation (Hétéro): NA%\n"

$`3`
[1] "Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 3 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nses_ethnicityOther\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"lifestyle_goFishingFreq_numeric\", \"ses_age\", \"ses_ethnicityWhite\", \"ses_immigrant\")\n\nCe cluster est composé de \nFemmes: NA%, \n     Immigrants: NA%\nÂge: NA ans, \n     Revenu: 109883.7 k$\nLangues: \n     En NA%, \n     Fr NA%, \n     Autres: 24.4%\nÉdu: NA (0=bas, 1=élevé), \n     Ethnie (Blanc): NA%\nOrientation (Hétéro): NA%\n"

$`4`
[1] "Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 4 :\nVariables avec une moyenne significativement plus élevée que la moyenne de la population en général :\nc(\"lifestyle_clothingStyleClassic\", \"lifestyle_goFishingFreq_numeric\", \"lifestyle_goHuntingFreq_numeric\", \"lifestyle_goMuseumsFreq_numeric\", \"lifestyle_motorizedActFreq_numeric\", \"lifestyle_smokeFreq\", \"lifestyle_volunteeringFreq_numeric\")\nVariables avec une moyenne significativement plus basse que la moyenne de la population en général :\nc(\"lifestyle_clothingStyleCasual\", \"lifestyle_ownPetNone\", \"ses_age\")\n\nCe cluster est composé de \nFemmes: NA%, \n     Immigrants: NA%\nÂge: NA ans, \n     Revenu: 122103.7 k$\nLangues: \n     En NA%, \n     Fr NA%, \n     Autres: 0.6%\nÉdu: NA (0=bas, 1=élevé), \n     Ethnie (Blanc): NA%\nOrientation (Hétéro): NA%\n"

In [5]:
reponses_openai <- list()
# Afficher ou traiter tous les prompts
for (cluster in names(prompts)) {
  # Par exemple, envoyer chaque prompt à OpenAI (si la fonction est correctement définie)
  conv <- openai_create_conversation(prompts[[cluster]])
  resp <- openai_chat_completion(conv)
  reponses_openai[[cluster]] <- resp$content
}


In [6]:
reponses_openai

$`1`
[1] "Pour une personne habitant au Québec, avec un profil tel que décrit, un prénom qui pourrait convenir est \"Chloé\". Voici une brève description du persona :\n\n**Nom :** Chloé\n\n**Âge :** Plus âgée que la moyenne générale de la population.\n\n**Situation économique :** Chloé dispose d'un revenu de 116 452 $ par an, ce qui est relativement élevé.\n\n**Langues :** Bien qu'elle vive au Québec, Chloé parle principalement l'anglais, ce qui est significativement plus fréquent que dans la population générale.\n\n**Éducation :** Son niveau d'éducation n'est pas spécifié, mais il est supposé être élevé, vu son revenu.\n\n**Origine ethnique :** Les données spécifiques ne sont pas disponibles, mais elle pourrait être perçue comme étant d'origine caucasienne.\n\n**Orientation sexuelle :** Non spécifiée.\n\n**Particularités :** Chloé se distingue par sa maîtrise de l'anglais dans un environnement majoritairement francophone, ce qui suggère qu'elle peut être impliquée dans des secteurs professionnels ou sociaux où l'anglais est prédominant.\n\nChloé pourrait être une professionnelle accomplie, peut-être dans un domaine international ou lié aux affaires, et elle est probablement bien intégrée dans des cercles anglophones tout en vivant au Québec."

$`2`
[1] "Pour une personne habitant au Québec, un prénom approprié pourrait être \"Élise.\"\n\nPersona 2 : Élise\n\nÉlise est une femme vivant au Québec, où elle parle principalement le français. Elle est plus âgée que la moyenne de la population générale et son revenu est élevé, à 104 135,7 k$. Élise ne parle pas souvent anglais et a une tendance nettement plus faible à participer à des activités de chasse et de bénévolat. En tant que francophone, elle est bien intégrée dans la culture québécoise, mais elle n'est pas très engagée dans les activités de plein air traditionnelles comme la chasse, ni dans le bénévolat communautaire. Sa présence dans le groupe des immigrants n'est pas précisée, mais elle s'identifie majoritairement à la culture francophone du Québec. Étant donné qu'elle est plus âgée, elle a probablement une stabilité financière et une certaine expérience de vie qui influencent son style de vie."

$`3`
[1] "Pour le prénom, je propose \"Kiana\". Ce prénom est à la fois unique et multiculturel, reflétant bien la diversité ethnique suggérée par les données. Voici une brève description pour le persona 3 :\n\n**Prénom : Kiana**\n\nKiana est une personne qui se distingue par sa riche diversité culturelle. Elle a des origines ethniques variées, ce qui se reflète dans ses intérêts et son mode de vie. Résidant au Québec, elle appartient à une tranche de revenu relativement élevée, avec une valeur moyenne de 109 883,7 $. Elle parle probablement plusieurs langues, ce qui lui permet de naviguer aisément dans des environnements multiculturels, bien que le français ne soit pas sa langue maternelle.\n\nElle n'est pas particulièrement intéressée par des activités telles que la pêche, et elle est plus jeune que la moyenne de la population générale. Contrairement à la population typique du Québec, Kiana n'est pas d'origine blanche majoritaire et n'est pas immigrante, ce qui suggère qu'elle peut être issue d'une minorité ethnique bien intégrée au Québec.\n\nKiana est sans doute bien éduquée, ce qui pourrait expliquer son revenu élevé. Elle pourrait travailler dans un domaine qui valorise ses compétences linguistiques et sa capacité à naviguer dans des contextes culturels variés, tels que les affaires internationales, la diplomatie, ou les médias. \n\nSa diversité culturelle et son style de vie urbain et sophistiqué font d'elle une citoyenne du monde bien ancrée dans la culture québécoise, tout en apportant sa propre touche unique à la communauté."

$`4`
[1] "Pour une personne habitant au Québec, je propose le prénom \"Émilie\". Voici une brève description pour le persona correspondant au profil 4 :\n\n**Émilie** est une personne active et engagée dans sa communauté. Elle a un style vestiment

In [7]:
description_clusters <- data.frame(prompt = unlist(prompts), description = unlist(reponses_openai))

In [8]:
description_clusters

,prompt,description
,<chr>,<chr>
1,"Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 1 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""ses_age"", ""ses_languageEnglish"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : Aucune variable avec une moyenne significativement plus basse que la moyenne de la population en général. Ce cluster est composé de Femmes: NA%, Immigrants: NA% Âge: NA ans, Revenu: 116452 k$ Langues: En NA%, Fr NA%, Autres: 0% Édu: NA (0=bas, 1=élevé), Ethnie (Blanc): NA% Orientation (Hétéro): NA%","Pour une personne habitant au Québec, avec un profil tel que décrit, un prénom qui pourrait convenir est ""Chloé"". Voici une brève description du persona : **Nom :** Chloé **Âge :** Plus âgée que la moyenne générale de la population. **Situation économique :** Chloé dispose d'un revenu de 116 452 $ par an, ce qui est relativement élevé. **Langues :** Bien qu'elle vive au Québec, Chloé parle principalement l'anglais, ce qui est significativement plus fréquent que dans la population générale. **Éducation :** Son niveau d'éducation n'est pas spécifié, mais il est supposé être élevé, vu son revenu. **Origine ethnique :** Les données spécifiques ne sont pas disponibles, mais elle pourrait être perçue comme étant d'origine caucasienne. **Orientation sexuelle :** Non spécifiée. **Particularités :** Chloé se distingue par sa maîtrise de l'anglais dans un environnement majoritairement francophone, ce qui suggère qu'elle peut être impliquée dans des secteurs professionnels ou sociaux où l'anglais est prédominant. Chloé pourrait être une professionnelle accomplie, peut-être dans un domaine international ou lié aux affaires, et elle est probablement bien intégrée dans des cercles anglophones tout en vivant au Québec."
2,"Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 2 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : c(""ses_age"", ""ses_languageFrench"") Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""lifestyle_goHuntingFreq_numeric"", ""lifestyle_volunteeringFreq_numeric"", ""ses_languageEnglish"") Ce cluster est composé de Femmes: NA%, Immigrants: NA% Âge: NA ans, Revenu: 104135.7 k$ Langues: En NA%, Fr NA%, Autres: 0.4% Édu: NA (0=bas, 1=élevé), Ethnie (Blanc): NA% Orientation (Hétéro): NA%","Pour une personne habitant au Québec, un prénom approprié pourrait être ""Élise."" Persona 2 : Élise Élise est une femme vivant au Québec, où elle parle principalement le français. Elle est plus âgée que la moyenne de la population générale et son revenu est élevé, à 104 135,7 k$. Élise ne parle pas souvent anglais et a une tendance nettement plus faible à participer à des activités de chasse et de bénévolat. En tant que francophone, elle est bien intégrée dans la culture québécoise, mais elle n'est pas très engagée dans les activités de plein air traditionnelles comme la chasse, ni dans le bénévolat communautaire. Sa présence dans le groupe des immigrants n'est pas précisée, mais elle s'identifie majoritairement à la culture francophone du Québec. Étant donné qu'elle est plus âgée, elle a probablement une stabilité financière et une certaine expérience de vie qui influencent son style de vie."
3,"Propose un prénom pour une personne habitant au Québec et donne une brève description pour le persona 3 : Variables avec une moyenne significativement plus élevée que la moyenne de la population en général : ses_ethnicityOther Variables avec une moyenne significativement plus basse que la moyenne de la population en général : c(""lifestyle_goFishingFreq_numeric"", ""ses_age"", ""ses_ethnicityWhite"", ""ses_immigrant"") Ce cluster est composé de Femmes: NA%, Immigrants: NA% Âge: NA ans, Revenu: 109883.7 k$ Langues: 

In [10]:
write.csv(description_clusters, "description_clusters.csv", row.names = FALSE)

In [9]:
saveRDS(description_clusters, file = "../../_PrivateFolder_datagotchi_federal_2025/data/clustering/can2025/03_pilot_clusters_description.rds")